In [3]:
from matplotlib import pyplot as plt
%matplotlib inline  
import seaborn as sns
import math
import cv2
import numpy as np

In [4]:

sns.set_theme(style="white", palette="pastel")

In [5]:
def edgeDetectSobel(path):
    img = cv2.imread(f'{path}', cv2.IMREAD_COLOR)

    # Convert to graycsale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.cvtColor(img_gray, cv2.COLOR_BGR2RGB)

    # Blur the image for better edge detection
    img_blur = cv2.GaussianBlur(img_gray, (3,3), 0)

    # Sobel Edge Detection
    sobelx = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
    sobely = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
    sobelxy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection
    
    # Canny Edge Detection
    edges = cv2.Canny(image=img_blur, threshold1=100, threshold2=200) # Canny Edge Detection
    edges = cv2.cvtColor(edges, cv2.COLOR_BGR2RGB)

    return edges, sobelx, sobely, sobelxy, img


In [6]:
def edgeDetectLaplace(path):
    ddepth = cv2.CV_16S
    kernel_size = 3
    
    im=cv2.imread(f'{path}', cv2.IMREAD_COLOR)
    if im is None:
        print ('Error opening image')
        print ('Program Arguments: [image_name -- default lena.jpg]')
        return -1
    # Remove noise by blurring with a Gaussian filter
    #im = cv2.GaussianBlur(im, (3, 3), 0)
    # Laplace on a gray scale picture
    img_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # Blur the image for better edge detection
    img_blur = cv2.GaussianBlur(img_gray, (3,3), 0)

    img_blur = cv2.cvtColor(img_gray, cv2.COLOR_BGR2RGB)

    # Apply Laplace function
    dst = cv2.Laplacian(img_blur, ddepth, ksize=kernel_size)
    
    # converting back to uint8
    colorlaplace = cv2.convertScaleAbs(dst)


    return colorlaplace




In [7]:
#path to images, may be different depending and where they are saved on your machine
path = 'images/dsnamed/RO09.jpg'
edge = edgeDetectSobel(path)
edge2 = edgeDetectLaplace(path)
canny = edge[0]
sobelx = edge[1]
sobely = edge[2]
sobelxy = edge[3]
original = edge[4]
laplace = edge2

In [8]:

def test1():
    f, ax = plt.subplots(1,6)
    f.set_figheight(25)
    f.set_figwidth(25)

    ax[0].imshow(canny)
    ax[0].set_title('Canny')

    ax[1].imshow(sobelx)
    ax[1].set_title('SobelX')

    ax[2].imshow(sobely)
    ax[2].set_title('SobelY')

    ax[3].imshow(sobelxy)
    ax[3].set_title('SobelXY')

    ax[4].imshow(laplace)
    ax[4].set_title('Laplace')

    ax[5].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    ax[5].set_title('Original')




In [9]:
def main(PATH):

    #start streaming video
    vs = cv2.VideoCapture(PATH)
    
    while True:
        ret,frame = vs.read()
        cv2.startWindowThread()
        if not ret:
            break
        (h, w) = frame.shape[:2] #w:image-width and h:image-height
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        img_blur = cv2.GaussianBlur(img, (3,3), 0)
        edge = cv2.Canny(img_blur,300,350)
        
        lines =cv2.HoughLinesP(edge, rho = 1, theta = 1*np.pi/180, threshold = 100, minLineLength = 170, maxLineGap = 10)
   
        try:
            for i in lines:
                x1,x2,y1,y2 = i[0]
                cv2.line(frame,(x1,x2), (y1,y2), (0,255,0),3)
        except:
            cv2.putText(frame, 'No Line Detected', (h//2, w//2), cv2.FONT_HERSHEY_SIMPLEX,5, (0, 0, 255), 20, cv2.LINE_AA)

        try:
            cv2.imshow("media", frame)
        except:
            break

        cv2.waitKey(1)

    vs.release()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [11]:
main('Sample_Road_Videos/Road5.mp4')